# *Dynamic Routing*
<img src="https://www.kvc.org/wp-content/uploads/2015/08/brain-600px-600x372.jpg" width="320" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [1]:
import pathlib

import IPython
import ipywidgets as widgets

import np_logging
import np_session
import np_services
import np_workflows
from np_workflows import npxc
import np_workflows.experiments.dynamic_routing as DR
import npc_shields

from np_services.resources.zro import ZroError 
import contextlib

np_logging.getLogger()

np_workflows.elapsed_time_widget()

Failed to enable remote-to-remote symlink creation: try running as admin


***
## Quiet mode
on  
*default*
- exception details are hidden
- regular messages displayed (log level = INFO)

off
- full exception details with traceback
- extra messages displayed (log level = DEBUG)

In [2]:
np_workflows.quiet_mode_widget()

ToggleButton(value=True, button_style='info', description='Quiet mode is on', icon='check', tooltip='Quiet mod…

***
## Launch apps via RSC
[optional]

In [3]:
import contextlib
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [4]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Select HAB

In [5]:
selection = DR.workflow_select_widget(mouse)

***
## Generate new session
Re-run this cell if workflow is changed above.

Check mouse ID and session are correct: this cell will lock them in!

In [6]:
experiment: np_workflows.DynamicRoutingExperiment = DR.new_experiment(mouse, user, selection.workflow)
session: np_session.Session = experiment.session

15:12 | np_workflows.shared.base_experiments | INFO | Hab(DRpilot_708019_20240322) created
15:12 | web | INFO | Hab(DRpilot_708019_20240322) created


***
## Set task
- select a preset task variant
- modify in the text box as needed (right) - hit enter to apply

Double-check task until we have confidence in the new widget!

In [7]:
experiment.use_github = False
experiment.task_name = "pupil test"

task_name = 'pupil test' doesn't correspond to a preset value, but the attribute is updated anyway!


In [8]:
import functools
run_pupil_test = functools.partial(experiment.run_script, "pupil test")
experiment.pupil_test_params = {'taskScript': 'TaskControl.py', 'subjectName': str(experiment.mouse), 'rigName': str(self.rig).replace('.',''), 'taskVersion':'pupil test'}

NameError: name 'self' is not defined

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [9]:
assert experiment.task_name, "task name has not been set - assign manually in cell above"
experiment.initialize_and_test_services()

15:12 | np_services.utils | INFO | mousedirector is running on W10DTM714204
15:12 | np_services.proxies | INFO | Proxy(MouseDirector) initialized: ready for use
15:12 | np_services.utils | INFO | sync_device is running on W10DTSM118309
15:12 | np_services.proxies | INFO | Proxy(Sync) initialized: ready for use
15:12 | np_services.proxies | INFO | VideoMVR initialized: ready for use
15:13 | np_services.proxies | INFO | ImageMVR initialized: ready for use
15:13 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
15:13 | np_services.proxies | INFO | NewScaleCoordinateRecorder | Test passed
15:13 | np_services.utils | INFO | camstim_agent is running on W10DT713941
15:13 | np_services.utils | INFO | phidget_server is running on W10DT713941
15:13 | np_services.proxies | INFO | Proxy(ScriptCamstim) initialized: ready for use


***
## Start devices recording

In [10]:
experiment.start_recording()

15:16 | np_services.proxies | INFO | Sync | Starting recording
15:17 | np_services.proxies | INFO | Sync | Verified: file on disk is increasing in size
15:17 | np_services.proxies | INFO | VideoMVR | Starting recording
15:17 | np_services.proxies | INFO | VideoMVR | Verified: 3 cameras recording to disk


In [11]:
run_pupil_test()

AttributeError: 'Hab' object has no attribute 'pupil_test_params'

***
## Stop recording

In [12]:
experiment.stop_recording_after_stim_finished()

15:19 | np_workflows.shared.base_experiments | WARNING | Waiting additional 4 s for MVR to finish writing...
15:19 | np_services.proxies | INFO | VideoMVR | Stopped recording
15:19 | np_services.proxies | INFO | Sync | Stopped recording


***

## Finalize

In [ ]:
experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

### copy all raw dataexcept ephys to /allen 

In [ ]:
experiment.copy_data_files()

### run and save QC notebook in current working directory 

In [ ]:
with contextlib.suppress(Exception):
    import npc_sessions
    npc_sessions.write_qc_notebook(
        session_path_or_id=experiment.session.npexp_path.as_posix(),
        save_path=pathlib.Path.cwd(),
        ephys_record_node_dirs=tuple(
            next(path.glob('Record Node*')).as_posix()
            for path in np_services.OpenEphys.data_files
            if experiment.session.folder in path.name
            and 'surface' not in path.name
        ),
    )

In [ ]:
import np_jobs
experiment.copy_workflow_files()
# np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
# experiment.copy_ephys()

## Manually move ephys data to workgroups folder (with checksum verification)

### Drag and drop with right-click, and choose `TeraMove here`

For folders in A: and B: drives:

- main ephys recording into session folder:    
    `A:\<sessionID>` -> `\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot\<sessionID>`
    
- surface channels recording into the folder above:
    `A:\<sessionID>_surface_channels` -> `\\allen\programs\mindscope\workgroups\dynamicrouting\PilotEphys\Task 2 pilot`
    
Ensure `verify` is checked